# Advanced Querying Mongo

**⚠️ IMPORTANT: Limit your print to avoid infinite scrolling. Otherwise your
code will be lost between print lines. If working with lists do:**

```python
list(collection.find(query))[:5] #or a reasonably low number
```

Importing libraries and setting up connection

In [9]:
from pymongo import MongoClient
import pandas as pd
import time
client = MongoClient("localhost:27017")
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [10]:
client.list_database_names()

['admin', 'config', 'ironhack', 'local']

In [13]:
db = client["ironhack"]
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack')

In [15]:
db.list_collection_names()

['restaurants', 'companies']

In [14]:
c = db.get_collection("companies")
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

In [17]:
# c.find_one()
# I did all of this just to confirm that the connection was working. On with the lab.

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [21]:
#Name = 'Babelgum' 
babelgum = {"name":"Babelgum"}

filter_ = babelgum

# Show just name
projection = {"_id":0,"name":1}

# limit to 5
list(c.find(filter_, projection).limit(5))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [27]:
#number_of_employees > 5000
num_emp = {"number_of_employees":{"$gt":5000}}

filter_ = num_emp

# Show just name
projection = {"_id":0,"name":1}

# limit to 20, sort by the number of employess
list(c.find(filter_, projection).sort("number_of_employees",-1).limit(20))

[{'name': 'Siemens'},
 {'name': 'IBM'},
 {'name': 'Toyota'},
 {'name': 'PayPal'},
 {'name': 'Nippon Telegraph and Telephone Corporation'},
 {'name': 'Samsung Electronics'},
 {'name': 'Accenture'},
 {'name': 'Tata Consultancy Services'},
 {'name': 'Flextronics International'},
 {'name': 'Safeway'},
 {'name': 'Sony'},
 {'name': 'LG'},
 {'name': 'Ford'},
 {'name': 'Boeing'},
 {'name': 'Digital Equipment Corporation'},
 {'name': 'Nokia'},
 {'name': 'MItsubishi Electric'},
 {'name': 'MItsubishi Electric'},
 {'name': 'Bertelsmann'},
 {'name': 'Comcast'}]

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [28]:
# Founded >=2000 <= 2005
after_200 = {"founded_year":{"$gte":2000}}
before_2005 = {"founded_year":{"$lte":2005}}
founded = {"$and":[after_200,before_2005]}

filter_ = founded

# Show just name and founded_year
projection = {"_id":0, "name":1, "founded_year":1}

# limit to 10
list(c.find(filter_, projection).limit(10))

[{'name': 'Wetpaint', 'founded_year': 2005},
 {'name': 'Zoho', 'founded_year': 2005},
 {'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'Omnidrive', 'founded_year': 2005},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Helio', 'founded_year': 2005},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [32]:
# Valuation >= 100M
valuation = {"ipo.valuation_amount":{"$gte":100000000}}

"""
"ipo": {
    "valuation_amount": {
      "$numberLong": "104000000000"
    },

"""


# Founded before 2010
founded = {"founded_year":{"$lte":2010}}

filter_ = {"$and":[valuation,founded]}

# Show just name and ipo
projection = {"_id":0, "name":1, "ipo":1}

# limit to 10
list(c.find(filter_, projection).limit(2))


[{'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Twitter',
  'ipo': {'valuation_amount': 18100000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2013,
   'pub_month': 11,
   'pub_day': 7,
   'stock_symbol': 'NYSE:TWTR'}}]

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [37]:
# Less than 1000 employees
num_emp = {"number_of_employees":{"$lt":1000}}
# Founded before 2005
founded = {"founded_year":{"$lt":2005}}
filter_ = {"$and":[founded,num_emp]}

# Show just name, number of employees and year founded
projection = {"_id":0, "name":1, "number_of_employees":1, "founded_year":1}

# limit to 10, order by number of employees
list(c.find(filter_, projection).sort("number_of_employees",-1).limit(10))

[{'name': 'Infinera Corporation',
  'number_of_employees': 974,
  'founded_year': 2000},
 {'name': 'NorthPoint Communications Group',
  'number_of_employees': 948,
  'founded_year': 1997},
 {'name': '888 Holdings', 'number_of_employees': 931, 'founded_year': 1997},
 {'name': 'Forrester Research',
  'number_of_employees': 903,
  'founded_year': 1983},
 {'name': 'Webmetrics', 'number_of_employees': 900, 'founded_year': 1999},
 {'name': 'SonicWALL', 'number_of_employees': 900, 'founded_year': 1991},
 {'name': 'Cornerstone OnDemand',
  'number_of_employees': 881,
  'founded_year': 1999},
 {'name': 'Buongiorno', 'number_of_employees': 800, 'founded_year': 1999},
 {'name': 'Cvent', 'number_of_employees': 800, 'founded_year': 1999},
 {'name': 'ZoomInfo', 'number_of_employees': 800, 'founded_year': 2000}]

### 6. All the companies that don't include the `partners` field.

In [38]:
# Do not include partners
sans_partners = {"partners":{"$exists":False}}
filter_ = sans_partners

# Show just name
projection = {"_id":0, "name":1}

# limit to 10, order by number of employees
list(c.find(filter_, projection).limit(10))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [42]:
# Have jull type in "category_code"
null_cat = {"category_code": {"$type":"null"}}
filter_ = null_cat

# Show just name
projection = {"_id":0, "name":1,"category_code":1}

# limit to 10, order by number of employees
list(c.find(filter_, projection).limit(10))

[{'name': 'Collective', 'category_code': None},
 {'name': 'Snimmer', 'category_code': None},
 {'name': 'KoolIM', 'category_code': None},
 {'name': 'Level9 Media', 'category_code': None},
 {'name': 'VidKing', 'category_code': None},
 {'name': 'Drigg', 'category_code': None},
 {'name': 'SpaceTime', 'category_code': None},
 {'name': 'Touch Clarity', 'category_code': None},
 {'name': 'MMDAYS', 'category_code': None},
 {'name': 'Inside Group', 'category_code': None}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [43]:
# employess >=100 < 1000
ge_100 = {"number_of_employees":{"$gte":100}}
l_1000 = {"number_of_employees":{"$lt":1000}}
num_emp = {"$and":[ge_100,l_1000]}

filter_ = num_emp

# Show just name and number_of_employees
projection = {"_id":0, "name":1, "number_of_employees":1}

# limit to 10
list(c.find(filter_, projection).limit(10))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120}]

### 9. Order all the companies by their IPO price in a descending order.

In [45]:
# Show just name and ipo
projection = {"_id":0, "name":1, "ipo":1}

# limit to 10
list(c.find({}, projection).sort("ipo.valuation_amount",-1).limit(3))

[{'name': 'GREE',
  'ipo': {'valuation_amount': 108960000000,
   'valuation_currency_code': 'JPY',
   'pub_year': 2008,
   'pub_month': 12,
   'pub_day': 17,
   'stock_symbol': '3632'}},
 {'name': 'Facebook',
  'ipo': {'valuation_amount': 104000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 2012,
   'pub_month': 5,
   'pub_day': 18,
   'stock_symbol': 'NASDAQ:FB'}},
 {'name': 'Amazon',
  'ipo': {'valuation_amount': 100000000000,
   'valuation_currency_code': 'USD',
   'pub_year': 1997,
   'pub_month': 5,
   'pub_day': None,
   'stock_symbol': 'NASDAQ:AMZN'}}]

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [46]:
# No filter
filter_ = {}

# Show just name and number_of_employees
projection = {"_id":0, "name":1, "number_of_employees":1}

# limit to 10
list(c.find(filter_, projection).sort("number_of_employees",-1).limit(10))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [47]:
# Founded second semester
second_semester = {"founded_month":{"$gt":6}}
founded = second_semester

filter_ = founded

# Show just name and founded_month
projection = {"_id":0, "name":1, "founded_month":1}

# limit to 1000
list(c.find(filter_, projection).limit(1000))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12},
 {'name': 'Jingle Networks', 'founded_month': 9},
 {'name': 'LifeLock', 'founded_month': 8},
 {'name': 'Wesabe', 'founded_month': 12},
 {'name': 'SmugMug', 'founded_month': 11},
 {'name': 'Google', 'founded_month': 9},
 {'name': 'Skype', 'founded_month': 8},
 {'name': 'Pando Networks', 'founded_month': 7},
 {'name': 'Ikan', 'founded_month': 12},
 {'name': 'delicious', 'founded_month': 9},
 {'name': 'AllPeers', 'founded_month': 8},
 {'name': 'Wize', 'founded_month': 8},
 {'name': 'SellABand', 'founded_month': 8},
 {'name': 'iContact', 'founded_month': 7},
 {'name': 'MeeVee', 'foun

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.000

In [50]:
# Acquisition > 10M
acquisition = {"acquisition.price_amount":{"$gt":10000000}}

# Founded before 200
founded = {"founded_year":{"$lt":2000}}

filter_ = {"$and":[acquisition,founded]}

# Show just name and price_amount
projection = {"_id":0, "name":1, "acquisition.price_amount":1}

# limit to 10
list(c.find(filter_, projection).sort("acquisition.price_amount", -1).limit(10))


[{'name': 'BEA Systems', 'acquisition': {'price_amount': 8500000000}},
 {'name': 'Navteq', 'acquisition': {'price_amount': 8100000000}},
 {'name': 'Sun Microsystems', 'acquisition': {'price_amount': 7400000000}},
 {'name': 'Pixar', 'acquisition': {'price_amount': 7400000000}},
 {'name': 'LSI', 'acquisition': {'price_amount': 6600000000}},
 {'name': 'National Semiconductor',
  'acquisition': {'price_amount': 6500000000}},
 {'name': 'aQuantive', 'acquisition': {'price_amount': 6400000000}},
 {'name': 'Siebel Systems', 'acquisition': {'price_amount': 5850000000}},
 {'name': 'Sybase', 'acquisition': {'price_amount': 5800000000}},
 {'name': 'Affiliated Computer Services',
  'acquisition': {'price_amount': 5750000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [51]:
# Acquired after 2010
acquisition = {"acquisition.acquired_year":{"$gt":2010}}

filter_ = {"$and":[acquisition,founded]}

# Show just name and acquisition
projection = {"_id":0, "name":1, "acquisition":1}

# limit to 10
list(c.find(filter_, projection).sort("acquisition.price_amount", -1).limit(3))

[{'name': 'LSI',
  'acquisition': {'price_amount': 6600000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://dealbook.nytimes.com/2013/12/16/avago-to-buy-lsi-for-6-6-billion/?_r=0',
   'source_description': 'Avago to Buy LSI for $6.6 Billion',
   'acquired_year': 2013,
   'acquired_month': 12,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Avago Technologies',
    'permalink': 'avago-technologies'}}},
 {'name': 'National Semiconductor',
  'acquisition': {'price_amount': 6500000000,
   'price_currency_code': 'USD',
   'term_code': 'cash',
   'source_url': 'http://www.national.com/news/item/0,1735,1522,00.html',
   'source_description': 'TI to acquire National Semiconductor Complementary portfolios are foundation for growth',
   'acquired_year': 2011,
   'acquired_month': 4,
   'acquired_day': 4,
   'acquiring_company': {'name': 'Texas Instruments',
    'permalink': 'texas-instruments'}}},
 {'name': 'NetLogic Microsystems',
  'acquisition': {

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [55]:
filter_ = {}

# Show just name and founded_year
projection = {"_id":0, "name":1, "founded_year":1}

# limit to 10
list(c.find(filter_, projection).sort("founded_year", -1).limit(10))

[{'name': 'Wamba', 'founded_year': 2013},
 {'name': 'Gimigo', 'founded_year': 2013},
 {'name': 'Clowdy', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Pikk', 'founded_year': 2013},
 {'name': 'WhosCall', 'founded_year': 2013},
 {'name': 'SEOGroup', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [ ]:
# Your Code

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [ ]:
# Your Code

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [ ]:
# Your Code

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [ ]:
# Your Code

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [ ]:
# Your Code

### 20. All the companies that have been 'deadpooled' after the third year.

In [ ]:
# Your Code

**⚠️ Did you do this?**

```python
list(collection.find(query))[:5] #or a reasonably low number
```